<a href="https://colab.research.google.com/github/virf96/Chat-Bot/blob/master/DatosFaltantes_SustitucionModa_FeatureEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sustitución categoría más frecuente  ==> Feature-engine


### Qué es Feature-engine?

Feature-engine es una librería de Python que hemos creado para este curso. 

- Feature-engine incluye todas las técnicas de ingeniería de variables descritas en este curso
- Feature-engine funciona como Scikit-learn, por lo tanto es fácil de aprender
- Feature-engine te permite implementar pasos de ingeniería de variables específicos para diferentes grupos de variables
- Feature-engine puede ser integrado con las pipelines de Scikit-learn pipeline permitiendo construir modelos fácilmente
** Feature-engine te permite diseñar y guardar un flujo de ingeniería de variables con procesos diseñados específicamente para diferentes grupos de variables.**

-------------------------------------------------------------------
Feature-engine puede ser instalado vía pip ==> pip install feature-engine

- Asegurate que haz instalado Feature-engine antes de correr este notebook

Para más detalle visita el [website the trainindata]( https://www.trainindata.com/feature-engine) 


## En este demo:

Vamos a usar **Feature Engine para hacer la sustitución por la categoría más frecuente** usando los datos Ames House Price.

### Nota: 
* 'Imputer' se deriva del verbo en inglés 'to impute' que quiere decir sustituir o reemplazar. Imputer es el objeto que completa la sustitución, de ahí el nombre dado a la clase.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install feature_engine

     |████████████████████████████████| 9.5MB 11.1MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

#  feature engine
from feature_engine import missing_data_imputers as mdi

In [5]:
# carguemos los datos con unas columnas seleccionadas

cols_to_use = [
    'BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt',
    'SalePrice'
]

data = pd.read_csv('/content/drive/My Drive/datasets/houseprice.csv', usecols=cols_to_use)
data.head()

,LotFrontage,MasVnrArea,BsmtQual,FireplaceQu,GarageYrBlt,SalePrice
0,65.0,196.0,Gd,NaN,2003.0,208500
1,80.0,0.0,Gd,TA,1976.0,181500
2,68.0,162.0,Gd,TA,2001.0,223500
3,60.0,0.0,TA,Gd,1998.0,140000
4,84.0,350.0,Gd,TA,2000.0,250000


In [7]:
#Valores faltantes
data.isnull().mean().sort_values(ascending=False)

FireplaceQu    0.472603
LotFrontage    0.177397
GarageYrBlt    0.055479
BsmtQual       0.025342
MasVnrArea     0.005479
SalePrice      0.000000
dtype: float64

Todas las variables predictivas tienen datos ausentes

In [8]:
# separar datos en segmentos entrenamiento y prueba

# primero, separemos el target (SalePrice) del resto de las variables
#Solo trabajaremo con el train set dado que solo debe aprender de este set
cols_to_use.remove('SalePrice')

X_train, X_test, y_train, y_test = train_test_split(data[cols_to_use],
                                                    data['SalePrice'],
                                                    test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((1022, 5), (438, 5))

## Feature-engine captura las variables categóricas automáticamente

In [9]:
# llamemos el imputer de feature engine
# no necesitamos especificar nada
#Indicamo el método "frequent" para que sea por moda
imputer = mdi.CategoricalVariableImputer(imputation_method='frequent')

In [11]:
# ajustemos el imputer
#Solo imputaremos a las variables categoricas de momento
imputer.fit(X_train)

CategoricalVariableImputer(fill_value='Missing', imputation_method='frequent',
                           return_object=False,
                           variables=['BsmtQual', 'FireplaceQu'])

In [12]:
# vemos que el imputer encontró las variables categóricas 
# para sustituir con la categoría más frecuente o moda


imputer.variables

['BsmtQual', 'FireplaceQu']

In [14]:
# aquí vemos los valores  que serán usados
# para reemplazar los NA en cada variable


imputer.imputer_dict_

{'BsmtQual': 'TA', 'FireplaceQu': 'Gd'}

In [15]:
# revisemos las modas sobre el set de entrenamiento

X_train[imputer.variables].mode()

,BsmtQual,FireplaceQu
0,TA,Gd


In [16]:
# feature-engine devuelve un dataframe

tmp = imputer.transform(X_train)
tmp.head()

,BsmtQual,FireplaceQu,LotFrontage,MasVnrArea,GarageYrBlt
64,Gd,Gd,NaN,573.0,1998.0
682,Gd,Gd,NaN,0.0,1996.0
960,TA,Gd,50.0,0.0,NaN
1384,TA,Gd,60.0,0.0,1939.0
1100,TA,Gd,60.0,0.0,1930.0


In [17]:
# revisemos que los valores nulos ya no existen

tmp[imputer.variables].isnull().mean()

BsmtQual       0.0
FireplaceQu    0.0
dtype: float64

## Feature-engine te permite especificar grupos de variables fácilmente

In [18]:
# usemos la sustitución pero esta vez solo 
# imputemos una variable

imputer = mdi.CategoricalVariableImputer(imputation_method='frequent',
                                         variables=['BsmtQual'])

imputer.fit(X_train)

CategoricalVariableImputer(fill_value='Missing', imputation_method='frequent',
                           return_object=False, variables=['BsmtQual'])

In [19]:
# ahora el imputer solo tiene la variable indicada

imputer.variables

['BsmtQual']

In [20]:
# y podemos ver el valor asignado para sustituirla

imputer.imputer_dict_

{'BsmtQual': 'TA'}

In [21]:
# feature-engine devuelve un dataframe
# al imputar la variable:

tmp = imputer.transform(X_train)


# revisemos que la variable indicada ya no tiene valores nulos
tmp[imputer.variables].isnull().mean()

BsmtQual    0.0
dtype: float64

Funcionó!

More details here:
https://feature-engine.readthedocs.io/en/latest/imputers/CategoricalVariableImputer.html